In [1]:
global center_lat, center_lon, center_heading



center_lat = 40.75794805357645
center_lon = -73.98553867868135
center_heading = 204.501

In [2]:
import numpy as np
import skimage as sk
import skimage.io as skio
import json
import copy
import triangle
import matplotlib.pyplot as plt
import skimage as ski
from PIL import Image
import imageio
import os
from tqdm import tqdm
import cv2
from scipy import signal
import numpy as np
import copy

In [3]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius_of_earth = 6371 * 1000  # Radius of Earth in kilometers. Use 3959 for miles.

    # Calculate the distance
    distance = radius_of_earth * c

    return distance

# # Example usage:
# lat1 = 37.76883408483024 # Latitude of the first point
# lon1 = -122.38892071430969  # Longitude of the first point
# lat2 = 37.76882967189774  # Latitude of the second point
# lon2 = -122.38884535002529  # Longitude of the second point

# distance = haversine(lat1, lon1, lat2, lon2)
# print(f"The distance between the two points is {distance:.2f} m.")

In [4]:
np.linspace(0, 1, 2)

array([0., 1.])

In [5]:
class Scene():
    def __init__(self, img_path, depth_path):
#         self.depth = skio.imread(depth_path)
#         self.depth = sk.img_as_float(self.depth)[:,:,0]
        data_array = np.genfromtxt(depth_path, delimiter=',', dtype=float)
        self.depth = data_array
        self.colors = skio.imread(img_path)
        self.colors = sk.img_as_float(self.colors)[:,:,:3]
        self.points = []
        self.w = self.colors.shape[1]
        self.h = self.colors.shape[0]
        self.selected_index  = []
        self.displacement = np.array([0,0,0])
        self.rotation_matrix = None
        img_path_split = img_path[:-4].split("_")
        self.lat = float(img_path_split[0])
        self.lon = float(img_path_split[1])
#         print(img_path_split)
        self.heading = float(img_path_split[2])
    
    def calculate_initial_compass_bearing(self, lat1, lon1, lat2, lon2):
        """
        Calculates the initial compass bearing between two points.

        Parameters:
        - lat1, lon1: Latitude and longitude of the starting point (in degrees)
        - lat2, lon2: Latitude and longitude of the destination point (in degrees)

        Returns:
        - Initial compass bearing in degrees (from 0 to 360)
        """

        # Convert latitude and longitude from degrees to radians
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Calculate the differences in longitude
        d_lon = lon2 - lon1

        # Calculate the initial bearing using atan2
        x = np.sin(d_lon) * np.cos(lat2)
        y = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1) * np.cos(lat2) * np.cos(d_lon))
        

        initial_bearing = atan2(x, y)

        # Convert the initial bearing from radians to degrees
        initial_bearing = initial_bearing/3.14159 * 180

        # Normalize the initial bearing to the range [0, 360)
        initial_bearing = (initial_bearing + 360) % 360

        return initial_bearing
    
    
    def calcualte_displacement(self, lat1=center_lat, lon1=center_lon,heading1=center_heading):
            lat2 = self.lat
            lon2 = self.lon
            heading2 = self.heading
            
            
            dheading = self.calculate_initial_compass_bearing(lat1, lon1, lat2, lon2)
            
            # Convert latitude and longitude from degrees to radians
            lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

            # Haversine formula
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            radius_of_earth = 6371 * 1000  # Radius of Earth in kilometers. Use 3959 for miles.

            # Calculate the distance
            distance = radius_of_earth * c
            print('distance', distance)

            
            print('dheading', dheading)
            print('heading2', heading2)
            dheading = dheading/180 * 3.14159
            heading2 = heading2/180*3.14159

            
            rotation_matrix = np.array([[np.cos(heading2),-np.sin(heading2),0],
                           [np.sin(heading2),np.cos(heading2),0],
                           [0,0,1]])

            self.rotation_matrix = rotation_matrix
            self.displacement = [-np.cos(dheading)*distance, np.sin(dheading)*distance, 0]
#             self.displacement = [np.sin(dheading)*distance, np.cos(dheading)*distance, 0]    

    
#     for offset_i in np.linspace(0, 1, 5):
#     for offset_j in np.linspace(0, 1, 5):
#         phi = (w-(j+offset_j)-1)/(w-1) * 2* 3.14159 + 0.5*3.14159
#         theta = (h-(i+offset_i)-1)/(h-1) * 3.14159
    
    def calculate_xyz_coordinate(self):
        self.calcualte_displacement()
        total_count = 0
        for i in range(self.h):
            for j in range(self.w):
                if self.depth[i,j] < 1e7:
                    for offset_i in [0, 0.33,0.66]:
                        for offset_j in [0, 0.33,0.66]:
                            phi = (1-(self.w-(j+offset_j)-1)/(self.w-1)) * 2* 3.14159 - 3.14159
                            theta = (self.h-(i+offset_i)-1)/(self.h-1) * 3.14159

    #                         phi = (1-(self.w-j-1)/(self.w-1)) * 2* 3.14159 - 3.14159
    #                         theta = (self.h-i-1)/(self.h-1) * 3.14159
                            v = np.array([np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta)])

            #                 if total_count == 39596:
            #                     print(v,self.depth[i,j])
                            v = v @ self.rotation_matrix
            #                     if total_count == 39596:
            #                         print(copy.copy(v*self.depth[i,j])
                            self.points.append(copy.copy(v*self.depth[i,j]) + self.displacement)
                            self.selected_index.append(i*self.w + j)
                total_count+=1
    
    def calculate_xyz_coordinate(self):
        self.calcualte_displacement()
        total_count = 0
        for i in range(self.h):
            for j in range(self.w):
                if self.depth[i,j] < 1e7:
                    phi = (1-(self.w-j-1)/(self.w-1)) * 2* 3.14159 - 3.14159
                    theta = (self.h-i-1)/(self.h-1) * 3.14159

#                         phi = (1-(self.w-j-1)/(self.w-1)) * 2* 3.14159 - 3.14159
#                         theta = (self.h-i-1)/(self.h-1) * 3.14159
                    v = np.array([-np.sin(theta)*np.cos(phi), -np.sin(theta)*np.sin(phi), np.cos(theta)])

    #                 if total_count == 39596:
    #                     print(v,self.depth[i,j])
                    v = v @ self.rotation_matrix
    #                     if total_count == 39596:
    #                         print(copy.copy(v*self.depth[i,j])
                    self.points.append(copy.copy(v*self.depth[i,j]) + self.displacement)
                    self.selected_index.append(i*self.w + j)
                total_count+=1
    

In [6]:


im_dir = './'
images = []
img_paths = []
# iterate over images in the directory
for f in os.scandir(im_dir):
    if f.is_file() and f.name.lower().endswith('.png'):
        img_paths.append(f.path[2:])
#         images.append(cv2.imread(f.path, cv2.IMREAD_COLOR))

# img_paths = ["40.75794805357645_-73.98553867868135_204.50131.png",
#              "40.75812700394543_-73.98544263890469_208.43321.png",
#             "40.758204963957915_-73.9853870416264_208.63332.png",
#             "40.75828229336389_-73.98533018890905_209.31956.png",
#             "40.7583595854157_-73.98527360519417_208.13182.png",
#             "40.75843655632809_-73.98522007034872_207.89609.png",
#             "40.75851027726113_-73.98516859797068_207.93457.png"]

# color_paths = ["37.768900652202035_-122.38887243223877_264.93262_depth.png"]
# scenes = []

# img_paths = ["37.78830013768426_-122.40846027989743_170.83008.png",
#              "37.78833694542047_-122.40824689775292_80.585556.png",
#             "37.78838501720517_-122.40790629781468_79.82836.png",
#             "37.788443762709406_-122.40744772635011_80.58542.png",
#             "37.788470763148744_-122.40722863807088_80.80982.png",
#             "37.78850309159115_-122.40696927498523_80.36718.png",
#             "37.78852089418571_-122.40682243768639_81.20509.png",
#             "37.78819031373786_-122.40672391225024_170.84293.png",
#              "37.78803952488978_-122.40669487315671_171.35109.png",

#             ]

depth_paths = []

for i in range(len(img_paths)):
    temp = img_paths[i][:-4].split("_")
    temp = "_".join(temp) + ".csv"
    depth_paths.append(temp)
depth_paths

['40.75778219278097_-73.98565525350915_209.27037.csv',
 '40.75828229336389_-73.98533018890905_209.31956.csv',
 '40.75843655632809_-73.98522007034872_207.89609.csv',
 '40.75747371868198_-73.985879615017_208.99036.csv',
 '40.7583595854157_-73.98527360519417_208.13182.csv',
 '40.7576839941884_-73.98487527689113_298.41705.csv',
 '40.75764187077263_-73.98477286997644_298.37045.csv',
 '40.75770365832646_-73.98571318220495_209.24088.csv',
 '40.758204963957915_-73.9853870416264_208.63332.csv',
 '40.75794805357645_-73.98553867868135_204.50131.csv',
 '40.75831113967238_-73.9863464511112_297.26767.csv',
 '40.75825022696765_-73.98619077942413_299.3132.csv',
 '40.75812700394543_-73.98544263890469_208.43321.csv',
 '40.75755712243159_-73.98456877337065_298.55942.csv',
 '40.758155549193695_-73.98599583094679_300.05267.csv',
 '40.75820320384766_-73.98609420193732_304.54727.csv',
 '40.75851027726113_-73.98516859797068_207.93457.csv']

In [7]:
#              "37.787599568596534_-122.40665511724181_261.94516.png",

In [8]:
# (-122.40846027989743 + 122.40665511724181) * 20037508/180

In [9]:
# import numpy as np

In [10]:
# np.log(np.tan((90+37.78830013768426) * 3.1459/360 ))/ 3.14159 * 20037508.34 - np.log(np.tan((90+37.787599568596534) * 3.1459/360 ))/ 3.14159 * 20037508.34

In [11]:
# haversine(37.787375776479095, -122.40826668898664, 37.787599568596534, -122.40665511724181)

In [12]:
scenes = []
for img_path, depth_path in zip(img_paths, depth_paths):
    temp = Scene(img_path, depth_path)
    scenes.append(copy.deepcopy(temp))

In [13]:
for scene in scenes:
    scene.calculate_xyz_coordinate()

distance 20.893744415768005
dheading 208.03028217677058
heading2 209.27037
distance 41.105536619699436
dheading 25.290263661772315
heading2 209.31956
distance 60.58627576750001
dheading 26.290838007633
heading2 207.89609
distance 60.054260047319865
dheading 208.56605110995503
heading2 208.99036
distance 50.91627451780336
dheading 26.007614095451572
heading2 208.13182
distance 63.121566106619206
dheading 117.72090802492994
heading2 298.41705
distance 72.93597735243891
dheading 117.82614732249823
heading2 298.37045
distance 30.895620333981277
dheading 208.40685462170774
heading2 209.24088
distance 31.2922332499611
dheading 24.08870869468427
heading2 208.63332
distance 0.0
dheading 0.0
heading2 204.50131
distance 79.11350185589622
dheading 300.6854698006688
heading2 297.26767
distance 64.38694121945255
dheading 301.4564064066561
heading2 299.3132
distance 21.479752963707714
dheading 22.12289770667337
heading2 208.43321
distance 92.53815139133046
dheading 118.01768682691932
heading2 298.55

In [14]:
points = []
colors = None

In [15]:
print(type(None))

<class 'NoneType'>


In [16]:
for scene in scenes:
    temp_points = scene.points
#     print(temp_points)
    points += temp_points
    temp_colors = scene.colors
    temp_colors = temp_colors.reshape(-1,3)
    temp_colors = temp_colors[scene.selected_index]
#     temp_colors = list(temp_colors)
#     print(temp_colors)
    if colors is None:
        colors = temp_colors
    else:
        colors =  np.concatenate((colors, temp_colors),axis=0)
#     print(type(temp_colors))

In [17]:
points = np.array(points)
print(points.shape)
print(colors.shape)

(1617682, 3)
(1617682, 3)


In [18]:
print(np.max(points[:,0])-np.min(points[:,0]))

273.64588775792134


In [19]:
print(np.max(points[:,1])-np.min(points[:,1]))

251.7219008863671


In [20]:
print(np.max(points[:,2])-np.min(points[:,2]))

58.22688659679802


In [21]:
# distances = np.linalg.norm(points[:, np.newaxis, :] - points, axis=2)

# # Set diagonal elements to zero to avoid comparing a point to itself
# np.fill_diagonal(distances, 0)

# # Find the maximum distance
# max_distance = np.max(distances)

In [22]:
import open3d as o3d
import numpy as np

# Define a list of points (replace this with your own coordinates)
# points = np.array(points)
# Create a point cloud
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0, origin=[0, 0, 0])
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(points)
point_cloud.colors = o3d.utility.Vector3dVector(colors)

print("Statistical oulier removal")
cl, ind = point_cloud.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=2.0)

flip_y_matrix = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, -1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]], dtype=np.float64)


# Apply the transformation to the point cloud
point_cloud.transform(flip_y_matrix)
# o3d.io.write_point_cloud("times_square.ply", point_cloud)

# point_clouds = []
# for i in range(-5,5):
#     for j in range(-5, 5):
#         points_2 = copy.copy(points)
#         points_2[:,0] += 0.1 * i
#         points_2[:,1] += 0.1 * j
#         point_cloud_2 = o3d.geometry.PointCloud()
#         point_cloud_2.points = o3d.utility.Vector3dVector(points_2)
#         point_cloud_2.colors = o3d.utility.Vector3dVector(colors)
#         point_clouds.append(copy.deepcopy(point_cloud_2))

# kdtree = o3d.geometry.KDTreeFlann(point_cloud)
# distances, _ = kdtree.search_knn_vector_3d(point_cloud.points, k=2)  # k=2 to get the distances to the two nearest neighbors

# # Extract the farthest neighbor distance for each point
# farthest_distances = np.max(distances, axis=1)

# # Print the maximum distance
# max_distance = np.max(farthest_distances)
# print("Maximum Distance:", max_distance)


# point_cloud.estimate_normals()
# # print('run Poisson surface reconstruction')
# with o3d.utility.VerbosityContextManager(
#         o3d.utility.VerbosityLevel.Debug) as cm:
#     mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
#         point_cloud, depth=9)
# vertices_to_remove = densities < np.quantile(densities, 0.05)
# mesh.remove_vertices_by_mask(vertices_to_remove)
# # print(mesh)
# o3d.visualization.draw_geometries([mesh,point_cloud])



# Visualize the point cloud



o3d.visualization.draw_geometries([mesh_frame, point_cloud] )
# o3d.visualization.draw_geometries([], window_name="Open3D", width=800, height=600)

Statistical oulier removal
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
